## Team 14: Eyes on the Sky

**Sean McElroy, Abraham Amoako-Atta, Sam Pritchard**

The following code pulls Real-Stream Data from NY2O, pulling data 17 of the most-tracked satellites. The product projects the satellites locations around the globe and gives in depth statistics, including how far the satallite currently is from Williamsburg.

*This code was adapted from the Arlington Bike Real-Stream demo from in class. Through vibe coding, we were able to adapt much of the script to fit our project. Changes such as the API Key and Satellite dictionary were our inputs*

**Import Necessary Libraries**

In [ ]:
# Importing Libraries
import requests
import json
import os
import time
from google.cloud import pubsub_v1

**Set up Connection with Google Cloud (using the same subscription/topic names as we did for the Arlington Bike data)**


In [ ]:
# Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../credentials/service_account.json"

# Set up Pub/Sub client and topic name
TOPIC_NAME = "projects/sjpritchardproject/topics/spin-bike-status"
publisher = pubsub_v1.PublisherClient()

**Personal API Key through NY2O, site used for getting the Real-Stream Satellite Data**


In [ ]:
# N2YO API key
API_KEY = 'VQAD7V-Z4W89L-UPZCZ4-5GDA

**Satellites we are tracking**


In [ ]:
# Dictionary of satellites (Name: NORAD ID)
SATELLITES = {
    "ISS": 25544,
    "SES 1": 36516,
    "NOAA 19": 33591,
    "GOES 13": 29155,
    "NOAA 15": 25338,
    "NOAA 18": 28654,
    "TERRA": 25994,
    "AQUA": 27424,
    "METOP-B": 38771,
    "SUOMI NPP": 37849,
    "GOES 15": 36411,
    "FOX-1A (AO-85)": 40967,
    "SAUDISAT 1C": 27607,
    "METEOR M2": 40069,
    "ASIASAT 3S": 25657,
    "NSS 12": 36032,
    "MEASAT 3B": 40147
}

**Location for which the data is being tracked from (Washington, USA)**


In [ ]:
# Location for tracking
LATITUDE = 47.6062
LONGITUDE = -122.3321
ALTITUDE = 0
DURATION = 2

**Fetches real-time satellite position data from the N2YO API for a given satellite ID. Returns a list of position objects (or an empty list on failure).**

In [ ]:
# Function to fetch satellite data
def fetch_satellite_data(sat_id):
    url = f"https://api.n2yo.com/rest/v1/satellite/positions/{sat_id}/{LATITUDE}/{LONGITUDE}/{ALTITUDE}/{DURATION}?apiKey={API_KEY}"
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
        print("[DEBUG] Raw API response:", json.dumps(data, indent=2))  # Debug print
        return data.get("positions", [])
    except Exception as e:
        print(f"[ERROR] Fetch failed: {e}")
        return []

**Publishes satellite position records to a Google Cloud Pub/Sub topic.**


In [ ]:
# Function to publish positions
def publish_positions(positions, sat_id):
    for pos in positions:
        # Validate required fields
        if "satlatitude" not in pos or "satlongitude" not in pos:
            print("[WARN] Skipping position due to missing lat/lon:", pos)
            continue

        record = {
            "satellite_id": sat_id,  # Now dynamic
            "lat": pos["satlatitude"],
            "lon": pos["satlongitude"],
            "altitude_km": pos.get("sataltitude", 0.0),
            "velocity_kms": pos.get("satvelocity", 0.0),  # May be missing in N2YO data
            "timestamp": pos.get("timestamp", int(time.time()))
        }

        # Debug print
        print("[DEBUG] Publishing record:", record)

        # Publish message to Pub/Sub topic
        publisher.publish(TOPIC_NAME, json.dumps(record).encode("utf-8"))

    print(f"[{time.strftime('%H:%M:%S')}] Published {len(positions)} satellite positions")

**Continuously looping through all satellites in the SATELLITES dictionary, fetching their latest position data from the N2YO API, and publishes that data to a Google Cloud Pub/Sub topic every 30 seconds.**

In [ ]:
# Main execution block
if __name__ == "__main__":
    while True:
        for name, sat_id in SATELLITES.items():
            print(f"[INFO] Tracking satellite: {name} (ID: {sat_id})")
            positions = fetch_satellite_data(sat_id)
            if positions:
                publish_positions(positions, sat_id)
            else:
                print(f"[{time.strftime('%H:%M:%S')}] No data fetched for {name}")
        time.sleep(10)  # Adjust if needed

## This is a real-time satellite tracking dashboard built using Dash, Plotly, Google Cloud Pub/Sub, and other visualization tools. It visualizes satellite positions, and proximity to a fixed location (Williamsburg, VA). 
### First we must import the necessary libraries and use google cloud credentials and the pubsub subscription.

In [ ]:
import os
import json
import time
import queue
import threading
import pandas as pd
from dash import Dash, dcc, html, Input, Output, callback_context
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash_bootstrap_components as dbc
import numpy as np
from datetime import datetime
from google.cloud import pubsub_v1
from math import radians, cos, sin, asin, sqrt

# Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../credentials/service_account.json"

# Pub/Sub subscription
SUBSCRIPTION_NAME = "projects/project-test-454816/subscriptions/spin-bike-status-sub"
client = pubsub_v1.SubscriberClient()
data_queue = queue.Queue()

### Then we can set Williamsburg as a reference point and set up the Satellite IDs to find certain satellites that are popular. We can assign colors and break up satellites with certain features.

In [ ]:
# Internal data store
sat_data = []

# Williamsburg, VA coordinates
WILLIAMSBURG_LAT = 37.2707
WILLIAMSBURG_LON = -76.7075
PROXIMITY_THRESHOLD = 500  # km - satellites within this distance are considered "nearby"

# Function to calculate distance between two points on Earth
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of earth in kilometers
    return c * r

# Satellite ID to Name map with improved descriptions
SATELLITE_NAMES = {
    25544: "ISS (International Space Station)",
    36516: "SES 1 (Communications)",
    33591: "NOAA 19 (Weather)",
    29155: "GOES 13 (Weather)",
    25338: "NOAA 15 (Weather)",
    28654: "NOAA 18 (Weather)",
    25994: "TERRA (Earth Observation)",
    27424: "AQUA (Earth Observation)",
    38771: "METOP-B (Weather)",
    37849: "SUOMI NPP (Weather)",
    36411: "GOES 15 (Weather)",
    40967: "FOX-1A (Amateur Radio)",
    27607: "SAUDISAT 1C (Communications)",
    40069: "METEOR M2 (Weather)",
    25657: "ASIASAT 3S (Communications)",
    36032: "NSS 12 (Communications)",
    40147: "MEASAT 3B (Communications)"
}

# Improved color palette for satellites
SATELLITE_COLORS = {
    25544: "#FF4136",  # ISS - bright red
    36516: "#0074D9",  # SES 1 - blue
    33591: "#2ECC40",  # NOAA 19 - green
    29155: "#FF851B",  # GOES 13 - orange
    25338: "#B10DC9",  # NOAA 15 - purple
    28654: "#01FF70",  # NOAA 18 - lime
    25994: "#FFDC00",  # TERRA - yellow
    27424: "#7FDBFF",  # AQUA - cyan
    38771: "#F012BE",  # METOP-B - magenta
    37849: "#39CCCC",  # SUOMI NPP - teal
    36411: "#3D9970",  # GOES 15 - olive
    40967: "#85144b",  # FOX-1A - maroon
    27607: "#AAAAAA",  # SAUDISAT 1C - gray
    40069: "#FF6F61",  # METEOR M2 - coral
    25657: "#6B5B95",  # ASIASAT 3S - lavender
    36032: "#88B04B",  # NSS 12 - moss green
    40147: "#EFC050"   # MEASAT 3B - gold
}

# Satellite types for filtering
SATELLITE_TYPES = {
    "Weather": [33591, 29155, 25338, 28654, 38771, 37849, 36411, 40069],
    "Communications": [36516, 27607, 25657, 36032, 40147],
    "Earth Observation": [25994, 27424],
    "Space Station": [25544],
    "Amateur Radio": [40967]
}



### Create backup data if necesarry and get subscription up and running.

In [ ]:
# Mock data generator for testing when no real data is available
def generate_mock_data():
    """Generate mock satellite data for testing when no real data is available"""
    mock_satellites = [25544, 36516, 33591]  # ISS, SES 1, NOAA 19
    
    for sat_id in mock_satellites:
        # Create mock position with slight movement
        lat = WILLIAMSBURG_LAT + (np.random.random() - 0.5) * 20
        lon = WILLIAMSBURG_LON + (np.random.random() - 0.5) * 20
        altitude = 400 + np.random.random() * 100
        velocity = 7.5 + np.random.random()
        
        timestamp = time.time()
        
        mock_data = {
            "satellite_id": sat_id,
            "lat": lat,
            "lon": lon,
            "altitude_km": altitude,
            "velocity_kms": velocity,
            "timestamp": timestamp,
            "received_time": time.strftime('%H:%M:%S', time.gmtime(timestamp)),
            "williamsburg_distance": haversine(WILLIAMSBURG_LON, WILLIAMSBURG_LAT, lon, lat),
            "is_mock": True  # Flag to indicate this is mock data
        }
        
        data_queue.put(mock_data)

# Subscriber callback
def callback(message):
    try:
        payload = json.loads(message.data)
        
        # Ensure timestamp exists and is in the right format
        if "timestamp" not in payload:
            payload["timestamp"] = time.time()
            
        payload["received_time"] = time.strftime('%H:%M:%S', time.gmtime(payload["timestamp"]))
        
        # Ensure all required fields are present
        required_fields = ["satellite_id", "lat", "lon", "altitude_km", "velocity_kms"]
        for field in required_fields:
            if field not in payload:
                if field in ["altitude_km", "velocity_kms"]:
                    payload[field] = 0.0
                elif field == "satellite_id":
                    # Skip messages without satellite_id
                    print(f"Skipping message without satellite_id: {payload}")
                    message.ack()
                    return
                else:
                    # Skip messages without essential coordinates
                    print(f"Skipping message without {field}: {payload}")
                    message.ack()
                    return
        
        # Make sure satellite_id is an integer
        try:
            payload["satellite_id"] = int(payload["satellite_id"])
        except (ValueError, TypeError):
            print(f"Skipping message with invalid satellite_id: {payload}")
            message.ack()
            return
        
        # Calculate distance from Williamsburg
        payload["williamsburg_distance"] = haversine(
            WILLIAMSBURG_LON, WILLIAMSBURG_LAT, 
            payload["lon"], payload["lat"]
        )
        
        data_queue.put(payload)
        message.ack()
    except Exception as e:
        print(f"Error processing message: {e}")
        message.ack()  # Still acknowledge to avoid redelivery of problem messages

# Start subscriber in background thread
def subscriber_thread():
    streaming_pull_future = client.subscribe(SUBSCRIPTION_NAME, callback=callback)
    try:
        streaming_pull_future.result()
    except Exception as e:
        print(f"Subscriber thread error: {e}")

threading.Thread(target=subscriber_thread, daemon=True).start()

### Use dash with bootstrap to create the initial dashboard with the containers and general layout of the final product. Ensure the display is correct and that the live data is being processed from pubsub into the live dashboard.

In [ ]:
# Dash setup with Bootstrap
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
app.title = "🛰️ Satellite Live Tracker"

# Layout
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col([
            html.H2("🛰️ Satellite Live Tracker", className="mb-2 mt-4"),
            html.P("Real-time satellite tracking with Williamsburg, VA proximity alerts", className="text-muted"),
        ], width=9),
        dbc.Col([
            html.Div(id="last-update", className="text-info mt-4"),
            dbc.Button("Refresh Data", color="primary", id="refresh-btn", className="mt-2"),
        ], width=3, className="text-end"),
    ], className="mb-4"),
    
    # Main content
    dbc.Row([
        # Left sidebar
        dbc.Col([
            # Filters card
            dbc.Card([
                dbc.CardHeader("Filters"),
                dbc.CardBody([
                    html.P("Satellite Type:", className="mb-1"),
                    dbc.Checklist(
                        id="satellite-type-filter",
                        options=[
                            {"label": sat_type, "value": sat_type} for sat_type in SATELLITE_TYPES.keys()
                        ],
                        value=list(SATELLITE_TYPES.keys()),  # All selected by default
                        inline=True,
                    ),
                    html.Hr(),
                    html.P("Orbit Altitude:", className="mb-1"),
                    dcc.RangeSlider(
                        id="altitude-slider",
                        min=0,
                        max=2000,
                        step=50,
                        marks={0: '0', 500: '500', 1000: '1000', 1500: '1500', 2000: '2000'},
                        value=[0, 2000],
                    ),
                    html.Hr(),
                    # Add a toggle for mock data
                    dbc.Checkbox(
                        id="use-mock-data",
                        label="Use mock data if no real data available",
                        value=True
                    ),
                ]),
            ], className="mb-4"),
            
            # Williamsburg Proximity Alert
            dbc.Card([
                dbc.CardHeader("Williamsburg Proximity Alert"),
                dbc.CardBody([
                    html.Div(id="williamsburg-proximity")
                ]),
            ], className="mb-4"),
            
            # Selected Satellite Info
            dbc.Card([
                dbc.CardHeader("Selected Satellite Info"),
                dbc.CardBody([
                    html.Div(id="selected-sat-info")
                ]),
            ]),
        ], width=12, lg=3),
        
        # Main visualization area
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Tabs(id="tabs", value="map", children=[
                        dcc.Tab(label="🌍 Live Map", value="map"),
                        dcc.Tab(label="📊 Stats", value="stats"),
                        dcc.Tab(label="📈 Time Series", value="charts"),
                        dcc.Tab(label="🌐 3D Globe", value="globe"),
                    ]),
                    dcc.Graph(id="sat-visual", style={"height": "70vh"}),
                ]),
            ]),
        ], width=12, lg=9),
    ]),
    
    # Data refresh interval
    dcc.Interval(id="interval", interval=5000, n_intervals=0),
], fluid=True)

@app.callback(
    [
        Output("sat-visual", "figure"), 
        Output("williamsburg-proximity", "children"),
        Output("selected-sat-info", "children"),
        Output("last-update", "children")
    ],
    [
        Input("interval", "n_intervals"),
        Input("tabs", "value"),
        Input("satellite-type-filter", "value"),
        Input("altitude-slider", "value"),
        Input("refresh-btn", "n_clicks"),
        Input("sat-visual", "clickData"),
        Input("use-mock-data", "value")
    ]
)
def update_display(n, selected_tab, selected_types, altitude_range, n_clicks, click_data, use_mock):
    global sat_data
    
    # Process incoming data
    while not data_queue.empty():
        try:
            sat_data.append(data_queue.get())
        except Exception as e:
            print(f"Error processing data: {e}")
    
    # Keep only data from the last 10 minutes
    now = time.time()
    sat_data[:] = [s for s in sat_data if now - s["timestamp"] <= 600]
    
    # Generate mock data if needed
    if use_mock and not sat_data:
        print("No real data available. Generating mock data...")
        generate_mock_data()
        # Process mock data from queue
        while not data_queue.empty():
            try:
                sat_data.append(data_queue.get())
            except Exception as e:
                print(f"Error processing mock data: {e}")
    
    # Default figure
    empty_fig = go.Figure().update_layout(
        template="plotly_dark",
        paper_bgcolor="#222",
        plot_bgcolor="#222",
        title="No data available. Please check your data source."
    )
    
    if not sat_data:
        return (
            empty_fig, 
            "No satellite data received.",
            "No satellite selected", 
            f"Last update: {datetime.now().strftime('%H:%M:%S')}"
        )
    
    try:
        # Convert to DataFrame
        df = pd.DataFrame(sat_data)
        
        # Apply filters
        selected_sat_ids = []
        for sat_type in selected_types:
            selected_sat_ids.extend(SATELLITE_TYPES[sat_type])
        
        df = df[df["satellite_id"].isin(selected_sat_ids)]
        
        # Ensure altitude data is numeric
        df["altitude_km"] = pd.to_numeric(df["altitude_km"], errors="coerce").fillna(0)
        
        df = df[(df["altitude_km"] >= altitude_range[0]) & (df["altitude_km"] <= altitude_range[1])]
        
        if df.empty:
            return (
                empty_fig, 
                "No satellites match the current filters.", 
                "No satellite selected", 
                f"Last update: {datetime.now().strftime('%H:%M:%S')}"
            )
        
        # Get latest positions for each satellite
        latest_positions = df.groupby("satellite_id").last().reset_index()
        
        # Check for satellites near Williamsburg
        near_williamsburg = latest_positions[latest_positions["williamsburg_distance"] < PROXIMITY_THRESHOLD]
        
        if len(near_williamsburg) > 0:
            proximity_alert = [
                html.H5("🔔 Satellites Near Williamsburg!"),
                html.Ul([
                    html.Li([
                        f"{SATELLITE_NAMES.get(int(row['satellite_id']), 'Unknown')} - ",
                        html.Strong(f"{row['williamsburg_distance']:.1f} km away")
                    ]) for _, row in near_williamsburg.iterrows()
                ])
            ]
        else:
            proximity_alert = [
                "No satellites currently near Williamsburg, VA",
                html.Br(),
                html.Small("(within 500 km)")
            ]
        
        # Handle satellite selection from click data
        selected_sat_info = "Click on a satellite to see details"
        if click_data and "points" in click_data and len(click_data["points"]) > 0:
            point = click_data["points"][0]
            if "customdata" in point and point["customdata"] is not None:
                try:
                    sat_id = int(point["customdata"][0])
                    sat_rows = latest_positions[latest_positions["satellite_id"] == sat_id]
                    if not sat_rows.empty:
                        sat_data_point = sat_rows.iloc[0]
                        selected_sat_info = [
                            html.H5(SATELLITE_NAMES.get(sat_id, "Unknown Satellite")),
                            html.P([html.Strong("ID: "), f"{sat_id}"]),
                            html.P([html.Strong("Latitude: "), f"{sat_data_point['lat']:.4f}°"]),
                            html.P([html.Strong("Longitude: "), f"{sat_data_point['lon']:.4f}°"]),
                            html.P([html.Strong("Altitude: "), f"{sat_data_point['altitude_km']:.2f} km"]),
                            html.P([html.Strong("Velocity: "), f"{sat_data_point['velocity_kms']:.2f} km/s"]),
                            html.P([html.Strong("Distance to Williamsburg: "), 
                                  f"{sat_data_point['williamsburg_distance']:.2f} km"]),
                            html.P([html.Strong("Data Source: "), 
                                  "Mock Data" if sat_data_point.get("is_mock", False) else "Live API"])
                        ]
                except Exception as e:
                    selected_sat_info = f"Error displaying satellite info: {e}"
        
        # Create the selected visualization
        if selected_tab == "map":
            fig = create_map_view(df, latest_positions)
        elif selected_tab == "stats":
            fig = create_stats_view(latest_positions)
        elif selected_tab == "charts":
            fig = create_charts_view(df)
        elif selected_tab == "globe":
            fig = create_globe_view(df, latest_positions)
        else:
            fig = empty_fig
        
        # Last update info
        last_update = f"Last update: {datetime.now().strftime('%H:%M:%S')}"
        
        return fig, proximity_alert, selected_sat_info, last_update
        
    except Exception as e:
        import traceback
        print(f"Error in update_display: {e}")
        print(traceback.format_exc())
        return (
            empty_fig,
            f"Error: {str(e)}",
            "Error displaying satellite info",
            f"Last update: {datetime.now().strftime('%H:%M:%S')} (error)"
        )

### Create the map view of the satellites and include factors such as the trail to show movement and Williamsburg with a proximity circle. Add the current positions and display this map view onto the dashboard created above.

In [ ]:
def create_map_view(df, latest_positions):
    fig = go.Figure()
    
    # Add Williamsburg marker
    fig.add_trace(go.Scattermapbox(
        lat=[WILLIAMSBURG_LAT],
        lon=[WILLIAMSBURG_LON],
        mode="markers+text",
        marker=dict(size=12, color="yellow", symbol="star"),
        text=["Williamsburg, VA"],
        textposition="top right",
        name="Williamsburg, VA"
    ))
    
    # Add proximity circle (500 km around Williamsburg)
    # Create a circle of points around Williamsburg
    theta = np.linspace(0, 2*np.pi, 100)
    radius_in_degrees = PROXIMITY_THRESHOLD / 111  # rough conversion from km to degrees
    circle_lats = WILLIAMSBURG_LAT + radius_in_degrees * np.sin(theta) * 0.7  # adjust for latitude
    circle_lons = WILLIAMSBURG_LON + radius_in_degrees * np.cos(theta)
    
    fig.add_trace(go.Scattermapbox(
        lat=circle_lats,
        lon=circle_lons,
        mode="lines",
        line=dict(width=1, color="yellow"),
        name="Proximity Zone (500 km)",
        hoverinfo="skip"
    ))
    
    # Process each satellite
    for _, row in latest_positions.iterrows():
        sat_id = int(row["satellite_id"])
        sat_name = SATELLITE_NAMES.get(sat_id, f"Unknown Satellite {sat_id}")
        sat_color = SATELLITE_COLORS.get(sat_id, "#FFFFFF")
        sat_df = df[df["satellite_id"] == sat_id].sort_values("timestamp")
        
        if len(sat_df) <= 1:
            # For satellites with only one data point, still show the marker
            fig.add_trace(go.Scattermapbox(
                lat=[row["lat"]],
                lon=[row["lon"]],
                mode="markers+text",
                marker=dict(size=10, color=sat_color),
                text=[sat_name.split(" (")[0]],  # Show only the short name
                textposition="top right",
                name=sat_name,
                customdata=[[sat_id, sat_name]],  # Store satellite ID for click interactions
                hovertemplate=(
                    f"<b>{sat_name}</b><br>" +
                    "Latitude: %{lat:.4f}°<br>" +
                    "Longitude: %{lon:.4f}°<br>" +
                    f"Altitude: {row['altitude_km']:.1f} km<br>" +
                    f"Velocity: {row['velocity_kms']:.2f} km/s<br>" +
                    "<extra></extra>"
                )
            ))
            continue
            
        # Add satellite trail
        fig.add_trace(go.Scattermapbox(
            lat=sat_df["lat"],
            lon=sat_df["lon"],
            mode="lines",
            line=dict(width=2, color=sat_color),
            name=f"{sat_name} Trail",
            showlegend=False
        ))
        
        # Add current position marker
        fig.add_trace(go.Scattermapbox(
            lat=[row["lat"]],
            lon=[row["lon"]],
            mode="markers+text",
            marker=dict(size=10, color=sat_color),
            text=[sat_name.split(" (")[0]],  # Show only the short name
            textposition="top right",
            name=sat_name,
            customdata=[[sat_id, sat_name]],  # Store satellite ID for click interactions
            hovertemplate=(
                f"<b>{sat_name}</b><br>" +
                "Latitude: %{lat:.4f}°<br>" +
                "Longitude: %{lon:.4f}°<br>" +
                f"Altitude: {row['altitude_km']:.1f} km<br>" +
                f"Velocity: {row['velocity_kms']:.2f} km/s<br>" +
                "<extra></extra>"
            )
        ))
    
    fig.update_layout(
        mapbox=dict(
            style="carto-darkmatter",  # Use a reliable style that works
            zoom=1.8,
            center={"lat": 20, "lon": 0}
        ),
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(0,0,0,0.5)"
        )
    )
    
    return fig

### Create the stats view and add it to the dashboard created with bootstrap above. Add different parts like the type distribution and altitude distribution.

In [ ]:
def create_stats_view(latest_positions):
    # Make sure we have data
    if latest_positions.empty:
        empty_fig = go.Figure().update_layout(
            title="No data available for statistics",
            template="plotly_dark",
            paper_bgcolor="#222",
            plot_bgcolor="#222"
        )
        return empty_fig
    
    # Create three subplots
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"type": "pie"}, {"type": "xy"}],
               [{"type": "xy", "colspan": 2}, {}]],
        subplot_titles=("Satellite Types", "Altitude Distribution", "Velocity vs Altitude")
    )
    
    # Satellite type distribution
    sat_types = []
    sat_counts = []
    
    for sat_type, sat_ids in SATELLITE_TYPES.items():
        count = sum(1 for sat_id in latest_positions["satellite_id"] if sat_id in sat_ids)
        if count > 0:
            sat_types.append(sat_type)
            sat_counts.append(count)
    
    # Add satellite type distribution pie chart
    fig.add_trace(
        go.Pie(
            labels=sat_types,
            values=sat_counts,
            name="Satellite Types",
            textinfo="label+percent",
            hole=0.4
        ),
        row=1, col=1
    )
    
    # Add altitude distribution
    altitude_bins = [0, 200, 400, 600, 800, 1000, 1500, 2000]
    altitude_labels = [f"{low}-{high}" for low, high in zip(altitude_bins[:-1], altitude_bins[1:])]
    altitude_counts = [0] * (len(altitude_bins) - 1)
    
    for _, sat in latest_positions.iterrows():
        alt = sat["altitude_km"]
        for i in range(len(altitude_bins) - 1):
            if altitude_bins[i] <= alt < altitude_bins[i + 1]:
                altitude_counts[i] += 1
                break
    
    fig.add_trace(
        go.Bar(
            x=altitude_labels,
            y=altitude_counts,
            name="Altitude Distribution",
            marker_color="lightblue"
        ),
        row=1, col=2
    )
    
    # Add velocity vs altitude scatter
    fig.add_trace(
        go.Scatter(
            x=latest_positions["altitude_km"],
            y=latest_positions["velocity_kms"],
            mode="markers+text",
            marker=dict(
                size=12,
                color=[SATELLITE_COLORS.get(int(sat_id), "#FFFFFF") for sat_id in latest_positions["satellite_id"]]
            ),
            text=[SATELLITE_NAMES.get(int(sat_id), "Unknown").split(" (")[0] for sat_id in latest_positions["satellite_id"]],
            textposition="top center",
            name="Velocity vs Altitude",
            customdata=[[int(sat_id), SATELLITE_NAMES.get(int(sat_id), "Unknown")] for sat_id in latest_positions["satellite_id"]]
        ),
        row=2, col=1
    )
    
    fig.update_layout(
        title="Satellite Statistics",
        template="plotly_dark",
        paper_bgcolor="#222",
        plot_bgcolor="#222",
        height=800
    )
    
    return fig

### Create the charts view to show the top 5 satellies and their altitude.

In [ ]:
def create_charts_view(df):
    if df.empty:
        empty_fig = go.Figure().update_layout(
            title="No data available for time series",
            template="plotly_dark",
            paper_bgcolor="#222",
            plot_bgcolor="#222"
        )
        return empty_fig
    
    # Get unique satellite IDs with data
    sat_ids = df["satellite_id"].unique()
    
    fig = go.Figure()
    
    # Select top 5 satellites with most data points for clarity
    sat_counts = df["satellite_id"].value_counts()
    top_sats = sat_counts.nlargest(min(5, len(sat_counts))).index
    
    # Plot altitude time series
    for sat_id in top_sats:
        sat_df = df[df["satellite_id"] == sat_id].sort_values("timestamp")
        sat_name = SATELLITE_NAMES.get(int(sat_id), f"Unknown Satellite {sat_id}")
        sat_color = SATELLITE_COLORS.get(int(sat_id), "#FFFFFF")
        
        fig.add_trace(go.Scatter(
            x=sat_df["received_time"],
            y=sat_df["altitude_km"],
            name=f"{sat_name}",
            line=dict(color=sat_color)
        ))
    
    fig.update_layout(
        title="Satellite Altitude Time Series (Top 5 Satellites)",
        xaxis_title="Time (UTC)",
        yaxis_title="Altitude (km)",
        template="plotly_dark",
        paper_bgcolor="#222",
        plot_bgcolor="#222"
    )
    
    return fig

### Create the globe tab on the dashboard to show each satellites position and orbit train on a 360 interactable globe.

In [ ]:
def create_globe_view(df, latest_positions):
    fig = go.Figure()
    
    # Add Williamsburg marker
    fig.add_trace(go.Scattergeo(
        lat=[WILLIAMSBURG_LAT],
        lon=[WILLIAMSBURG_LON],
        mode="markers+text",
        marker=dict(size=8, color="yellow", symbol="star"),
        text=["Williamsburg, VA"],
        textposition="top right",
        name="Williamsburg, VA"
    ))
    
    # Process each satellite
    for _, row in latest_positions.iterrows():
        sat_id = int(row["satellite_id"])
        sat_name = SATELLITE_NAMES.get(sat_id, f"Unknown Satellite {sat_id}")
        sat_color = SATELLITE_COLORS.get(sat_id, "#FFFFFF")
        sat_df = df[df["satellite_id"] == sat_id].sort_values("timestamp")
        
        if len(sat_df) <= 1:
            # For satellites with only one data point, still show the marker
            fig.add_trace(go.Scattergeo(
                lat=[row["lat"]],
                lon=[row["lon"]],
                mode="markers+text",
                marker=dict(size=8, color=sat_color),
                text=[sat_name.split(" (")[0]],  # Show only the short name
                textposition="top right",
                name=sat_name,
                customdata=[[sat_id, sat_name]]
            ))
            continue
            
        # Add orbit trail
        fig.add_trace(go.Scattergeo(
            lat=sat_df["lat"],
            lon=sat_df["lon"],
            mode="lines",
            line=dict(width=2, color=sat_color),
            name=f"{sat_name} Orbit"
        ))
        
        # Add current position marker
        fig.add_trace(go.Scattergeo(
            lat=[row["lat"]],
            lon=[row["lon"]],
            mode="markers+text",
            marker=dict(size=8, color=sat_color),
            text=[sat_name.split(" (")[0]],  # Show only the short name
            textposition="top right",
            name=sat_name,
            customdata=[[sat_id, sat_name]]
        ))
    
    fig.update_geos(
        projection_type="orthographic",
        showland=True, landcolor="rgb(30, 40, 50)",
        showocean=True, oceancolor="rgb(20, 30, 70)",
        showcoastlines=True, coastlinecolor="rgb(60, 70, 80)",
        showcountries=True, countrycolor="rgb(100, 100, 100)"
    )
    
    fig.update_layout(
        title="3D Globe View",
        template="plotly_dark",
        paper_bgcolor="#222",
        plot_bgcolor="#222"
    )
    
    return fig

### Finally, run the app to create the dashboard with live satellite data.

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)